In [24]:
import omf
import pandas as pd
import numpy as np

In [36]:
csv_file = "E:\\MDA\\Beta Testing\\MIOPP\\Block Model - Mining Model\\reblocked_menkar_10x10x10.csv"
df = pd.read_csv(csv_file, skiprows=[1, 2])
df = df.sort_values(by=["EAST", "NORTH", "RL"], ascending=[True, True, True])
df

,EAST,_EAST,NORTH,_NORTH,RL,_RL,CLASSIFICATION,CO,CU,NI,DENSITY,DOMAIN,GEOTECH_ZONE,STRENGTH,WEATHER,EXCLUSION_ZONE,PERCENTAGE
0,295265.0,10.0,6796665.0,10.0,-395.0,10.0,4,0.001,0.001,0.001,2.5,ng,671088640,Strong,Fresh,0,1.0
4096,295265.0,10.0,6796665.0,10.0,-385.0,10.0,4,0.001,0.001,0.001,2.5,ng,671088640,Strong,Fresh,0,1.0
512,295265.0,10.0,6796665.0,10.0,-375.0,10.0,4,0.001,0.001,0.001,2.5,ng,671088640,Strong,Fresh,0,1.0
4608,295265.0,10.0,6796665.0,10.0,-365.0,10.0,4,0.001,0.001,0.001,2.5,ng,671088640,Strong,Fresh,0,1.0
64,295265.0,10.0,6796665.0,10.0,-355.0,10.0,4,0.001,0.001,0.001,2.5,ng,671088640,Strong,Fresh,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5878743,297555.0,10.0,6799495.0,10.0,455.0,10.0,4,0.000,0.000,0.000,0.0,ng,671088640,AIR,Air,0,0.0
5878750,297555.0,10.0,6799495.0,10.0,465.0,10.0,4,0.000,0.000,0.000,0.0,ng,671088640,AIR,Air,0,0.0
5878751,297555.0,10.0,6799495.0,10.0,475.0,10.0,4,0.000,0.000,0.000,0.0,ng,671088640,AIR,Air,0,0.0
5878798,297555.0,10.0,6799495.0,10.0,485.0,10.0,4,0.000,0.000,0.000,0.0,ng,671088640,AIR,Air,0,0.0


In [37]:
bm_origin = [6796660, 295260, -400]
min_y, max_y = 6796660, 6799500
min_x, max_x = 295260, 297560
min_z, max_z = -400, 500
block_size = 10.
differenceX = max_x - min_x
differenceY = max_y - min_y
differenceZ = max_z - min_z
num_blocksX = differenceX // block_size
num_blocksY = differenceY // block_size
num_blocksZ = differenceZ // block_size

tensor_u = np.full(int(num_blocksX), block_size, dtype=float)
tensor_v = np.full(int(num_blocksY), block_size, dtype=float)
tensor_w = np.full(int(num_blocksZ), block_size, dtype=float)

In [38]:
bm_attributes = []

for element in ["CU", "NI", "DENSITY", "CO"]:
  if element in df.columns:
      element_array = df[element].astype(float).to_numpy()
      attr = omf.ScalarData(
         array = np.array(element_array), name =element,  location='cells', )
      bm_attributes.append(attr)

In [39]:
proj = omf.Project(
    name='Menkar',
    description='Menkar OMF BM'
)
vol = omf.VolumeElement(
    name='Menkar BM',
    geometry=omf.VolumeGridGeometry(
        tensor_u=tensor_u,
        tensor_v=tensor_v,
        tensor_w=tensor_w,
        origin=bm_origin
    ),
    data=bm_attributes
)

In [40]:
proj.elements = [vol]
assert proj.validate()
omf.OMFWriter(proj, 'omfproj.omf')